# Detailed description of software protocol
## A fully reproducible one-stop-shop for the analysis of iTRAQ/TMT data

This file serves as documentation for the IsoProt protocol including explanation of the folder structure, details on the available parameters, the output files and instruction how to access the underlying code.

### Folders

This section describes the folder structure *inside* the docker container:

*IN*: This folder contains the files necessary to run the example, i.e. spectra file `iTRAQCancer.mgf`, human protein sequences (SwissProt) `sp_human.fasta` and the experimental design `exp_design_example.tsv`. If you want to use this folder for your own analysis, you need to **remove these example files** as they will otherwise be merged with your data.

*OUT*: Fixed output folder for all intermediate and end result file. This folder is linked to the _data_ folder and therefore can be accessed from outside the Docker container.

*data*: Starting IsoProt via the `run.sh` / `run.bat` scripts automatically maps the current folder (on the host) to `/data`. Therefore, to simplify the analysis we recommend to execute the `run.sh` / `run.bat` script directly in the directory containing all spectra and sequence database files. In case you manually launch the docker container, map your spectra directory to `/data` using the `-v` option (for example, `docker run -v /path/to/my/spectra/directory/:/data/ veitveit/isolabeledprotocol`).

*bin*: Here you can find the executables of [SearchGUI](http://compomics.github.io/projects/searchgui.html) and [PeptideShaker](http://compomics.github.io/projects/peptide-shaker.html).

*Scripts*: Python scripts to handle parameter selection, execution of tools and R scripts

### Parameters

#### Folders and database search
*Folder for spectra files (files need to be mgf) and fasta database:*

We allow the folder `IN` and `/data` where the latter is automatically mapped to the directory from which IsoProt is called (when using `run.sh` or `run.bat`). The folder should contain the spectra files and a database of protein sequences given in a FASTA file. 

Raw files (e.g. Thermo raw files) can be converted into mgf using the msconvert tool of [ProteoWizard](http://proteowizard.sourceforge.net/tools.shtml). This requires a Windows computer. Files from different MS runs should be organized in folders as illustrated in Figure 1.

![Illustration of folder structure](misc/ExperimentalDesigns.svg)
**Figure 1**: Illustration of experimental designs and folder structure.

#### Database search

*Fasta file*: Select from FASTA files in the above given folder.
The sequence headers of the file should be formatted in a [UniProt](http://uniprot.org)-like format. See more details on a suitable version of the FASTA format in the [SearchGUI documentation](https://github.com/compomics/searchgui/wiki/DatabaseHelp). Do not provide decoy sequences as they will be created automatically.

*Precursor tolerance (ppm)*: MS tolerance in parts per million

*Fragment ion tolerance (da)*: MSn tolerance in Dalton

*Number of miscleavages*

*Further fixed modifications*: Select Carbamidomethylation or None. iTRAQ and TMT labels as fixed modifications are added automatically based on the selected `Quantification method`

*Further variable modifications (Hold Ctrl to select multiple)*: Select variable modifications for the database search. Modified sequences will be used in the quantification and summarization of the proteins.

*Target (protein, peptide, and PSM) FDR*: False discovery rate for the peptide identifications.

*Quantification method*: Labelling method used in the experiment (iTRAQ/TMT)

*Number of different conditions*: Number of different types of samples (e.g. time points, treatments, cell types). This parameter defines how many sample groups will be generated in the `Experimental design` section.


#### Experimental design

This section is used to 1) assign sample names to each channel in each TMT/iTRAQ run, 2) assign labels for the sample groups (for example, "Treatment" and "Control", the number of sample groups is created based on the selection before) and 3) assign samples to sample groups. All default labels can be changed. The entire experimental design will be written into the table `exp_design.tsv` after pressing the `Save design` button. The file is located in the `OUT` folder.


#### Quantitative analysis

*Summarization method*: It is possible to select from the following methods to summarize peptide/PSM quantifications into protein abundance changes provided by the Bioconductor package [MSnbase](https://bioconductor.org/packages/release/bioc/html/MSnbase.html)

- iPQF: feature-based weighting of peptide spectra according to this [paper](https://www.ncbi.nlm.nih.gov/pubmed/26589272)
- Average over all PSMs (on log-scale)
- Median over all PSMs (on log-scale)
- Median over all PSMs after outlier removal (on log-scale)
- Robust summarization using the R function `rlm` (on log-scale)


*Minimum number of PSMs per protein*: Proteins or protein groups will only be quantified when 2 PSMs are available. This will be extended to unique peptides in the future

*Use PTMs or quantification*: Include peptides decorated by variable PTMs like oxidations in the protein summarization.

### Output files and figures

All output files are available in the `OUT` folder. For experiments with multiple runs, organised in subfolders, will report MS run specific results in corresponding subfolders of `OUT` and general results in `OUT`.

The database search via SearchGUI and validation via PeptideShaker result in a tab-delimited file (`experiment1_test_1_Extended_PSM_Report.txt`) containing detailed information from all peptide-spectrum matches (PSMs). Search and output parameters are automatically saved in json-formatted text file, `protocol_parameters.json`. Detailed information about identified mass spectra can be browsed importing the file `experiment.cpsx` to your local version of PeptideShaker.

Each spectra file is processed separately to match and quantify PSMs that passed the identification FDR (default 0.01). 

The mass distribution number of all matched fragment ions allows control for critical channels with inefficient labeling (*QC_ReporterIons_**filename**.pdf/png*). 
The output of all files of each run on the mass spectrometer are merged, normalized and visualized for quality control. Violin plots of median-normalized PSM intensities compare the intensity distributions (*QC_PSM_violinplots.pdf/png*). Channels with different distributions can identify problematic samples or changes within the entire proteome. All FDR-filtered and normalized PSM quantifications are saved to `AllQuantPSMs.csv` and `AllQuantPSMs.RData`. 

Six different histograms counting PSM, peptide, protein and protein group numbers allows determining protein coverage and uniqueness by the available mass spectra (`QC_PSM_and_peptide_distribution.pdf/png`). 

Similarity between samples is assessed through scatter plots comparing all the quantifications from all ion channels (`QC_Pairwise_comparison.pdf/png`).

Using the default parameters values, the PSMs are summarized to proteins by the iPQF summarization algorithm requiring a minimum of 1 PSM per protein, available through `AllQuantProteins.csv`. Other methods such as median summarization after outlier removal are available. 

A violin plot of protein ratios versus mean of all experimental conditions, available through `AllQuantProteinsInAllSamples.csv` shows should exhibit similar distributions for the different channels (`QC_Protein_violinplots.pdf/png`). 

Quantifications from different runs (only one in the example) are merged and submitted to principal component analysis. With the scoring plot, one can look for similarity within replicates compared to different experimental conditions (`QC_Stat_PCA.pdf/png`). Completely mixed samples are unlikely to provide differentially regulated proteins.

LIMMA statistical tests find regulated proteins.
p-values and false discovery rates (p-values corrected for multiple testing) for pairwise comparisons (here X versus Y), all available through `DifferentiallyRegulatedProteins.csv`, are visualized in histograms, volcano plots and a figure counting the number of differentially regulated proteins over a range of FDRs (*QC_Stat_Summary_**ConditionX-ConditionY**.pdf/png*). The latter can be used to identify a suitable combination of the confidence threshold and the number of significant proteins. It is advised to keep FDR<0.1 as the number of false positives becomes critically high otherwise.

### Code structure and scripts
#### Access

All python and R scripts are easily accessible in the Docker image. Python scripts to run parameter seection and database search can be found in the folder `Scripts`. R scripts for downstream analysis can be made visibile by going to `View->Cell Toolbar->Hide code` in the menu of the Isobaric_Workflow.ipynb Jupyter notebook. For the cells you want to see the code, uncheck the `Hide code` checkbox. 

